In [2]:
import numpy as np
import math
import glob
import os
import umap
import hdbscan
import seaborn as sns
import pandas as pd
import pickle
import openpyxl
import cv2
from collections import defaultdict

In [116]:
pilot3_rewardcomp = pd.read_excel(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\rewardcomp\iwata_pilot_3_reward_competition.xlsx")
pilot2_rewardcomp = pd.read_excel(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\rewardcomp\iwata_pilot_2_reward_competition.xlsx")
pilot1_rewardcomp = pd.read_excel(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\rewardcomp\cum_pilot_1_reward_competition.xlsx")


In [117]:
def extract_mouse1(match):
    return(extract_mice_identity(match)[0])
def extract_mouse2(match):
    return(extract_mice_identity(match)[1])

def extract_mice_identity(match):
    pieces = match.split(' ')
    if len(pieces) == 3:
        mouse1 = pieces[0]
        mouse2 = pieces[2]
    else:
        pieces = match.split("vs")
        if len(pieces) > 1:
            mouse1 = pieces[0]
            mouse2 = pieces[1]
        else:
            pieces = match.split("v")
            mouse1 = pieces[0]
            mouse2 = pieces[1]
    return(mouse1, mouse2)




In [123]:
def reward_comp_matrices(rewardcomp_df):
    matches = rewardcomp_df['Match'].values
    mouse1wins = rewardcomp_df['Mouse 1 Wins'].values
    mouse2wins = rewardcomp_df['Mouse 2 Wins'].values
    matrices = create_matrices(matches)
    full_matrices = fill_matrices(matches, mouse1wins, mouse2wins, matrices)
    full_matrices = list(full_matrices.values())
    return(full_matrices)

def create_matrices(matches):
    subjects = []
    for match in matches:
        pieces = match.split(' ')
        if len(pieces) == 3:
            subjects.append(pieces[0])
            subjects.append(pieces[2])
        else:
            pieces = match.split("vs")
            if len(pieces) > 1:
                subjects.append(pieces[0])
                subjects.append(pieces[1])
            else:
                pieces = match.split("v")
                subjects.append(pieces[0])
                subjects.append(pieces[1])
                
    subjects = np.unique(subjects)
    cages = defaultdict(list)
    for subject in subjects: 
        cages[subject[0]].append(subject)

    matrices = defaultdict()
    for cage, subjects in cages.items():
        matrices[cage] = pd.DataFrame(index = subjects, columns = subjects, data = 0)
    return(matrices)

def fill_matrices(matches, mouse1wins, mouse2wins, matrices):
    for i in range(len(matches)):
        pieces = matches[i].split(' ')
        if len(pieces) == 3:
            mouse1 = pieces[0]
            mouse2 = pieces[2]
        else:
            pieces = matches[i].split("vs")
            if len(pieces) > 1:
                mouse1 = pieces[0]
                mouse2 = pieces[1]
            else:
                pieces = matches[i].split("v")
                mouse1 = pieces[0]
                mouse2 = pieces[1]
        mouse1win = mouse1wins[i]
        mouse2win = mouse2wins[i]
        cage = mouse1[0]
        matrices[cage].loc[mouse1,mouse2] = mouse1win + matrices[cage].loc[mouse1,mouse2]
        matrices[cage].loc[mouse2,mouse1] = mouse2win + matrices[cage].loc[mouse2,mouse1]
    return(matrices)

 
rc_pilot3_matrices = reward_comp_matrices(pilot3_rewardcomp)
rc_pilot2_matrices = reward_comp_matrices(pilot2_rewardcomp)
rc_pilot1_matrices = reward_comp_matrices(pilot1_rewardcomp)




In [156]:
pilot1_hco = pd.ExcelFile(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\hco\iwata_pilot_1_home_cage_observations.xlsx")
pilot3_hco = pd.ExcelFile(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\hco\iwata_pilot_3_home_cage_observations.xlsx")
pilot2_hco = pd.ExcelFile(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\hco\iwata_pilot_2_home_cage_observations.xlsx")


def hco_matrices_pilot1(xlfile):
    matrices = []
    for sheet in xlfile.sheet_names:
        df = xlfile.parse(sheet)
        df.columns = df.columns.str.lower()
        df = df[(df["action"] == "f") | (df["action"] == "c")]
        matrices = make_matrices(df, matrices)
        
    return(matrices)

def make_matrices (df, matrices):
    groups = df.groupby(["winner", "loser"]).size().unstack(fill_value = 0)
    unique_subjects = set(groups.index).union(set(groups.columns))
    matrix = groups.reindex(index=sorted(unique_subjects),
                            columns=sorted(unique_subjects),
                            fill_value=0)
    matrices.append(matrix)
    return(matrices)

def hco_matrices(xlfile):
    matrices = []
    for sheet in xlfile.sheet_names:
        df = xlfile.parse(sheet,skiprows=1)
        df.columns = df.columns.str.lower()
        df = df[(df["action"] == "f") | (df["action"] == "c")]
        matrices = make_matrices(df, matrices)
    return(matrices)

def combine_matrices(matrices, pilot_num, combined = None):
    for matrix in matrices: 
        matrix.columns = matrix.columns.astype(str)
        matrix.index = matrix.index.astype(str)
        matrix.columns = ['*' + col[1:] for col in matrix.columns]
        matrix.index = [pilot_num +'.' + index for index in matrix.index]
    if combined is None:
        combined = pd.concat(matrices)
    else: 
        temp = pd.concat(matrices)
        combined = pd.concat([combined,temp])
    return(combined)
    
hco_pilot1_matrices = hco_matrices_pilot1(pilot1_hco)
hco_pilot2_matrices = hco_matrices(pilot2_hco)
hco_pilot3_matrices = hco_matrices(pilot3_hco)

hco_combined_matrices = combine_matrices(hco_pilot1_matrices, '1')
hco_combined_matrices = combine_matrices(hco_pilot2_matrices, '2', hco_combined_matrices)
hco_combined_matrices = combine_matrices(hco_pilot3_matrices, '3', hco_combined_matrices)


In [125]:
rc_combined_matrices = combine_matrices(rc_pilot1_matrices, '1')
rc_combined_matrices = combine_matrices(rc_pilot2_matrices, '2', rc_combined_matrices)
rc_combined_matrices = combine_matrices(rc_pilot3_matrices, '3', rc_combined_matrices)


In [148]:
pilot1_tt = pd.ExcelFile(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\tubetest\iwata_pilot_1_tube_test.xlsx")
pilot2_tt = pd.ExcelFile(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\tubetest\iwata_pilot_2_tube_test.xlsx")
pilot3_tt = pd.ExcelFile(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\tubetest\iwata_pilot_3_tube_test.xlsx")


def tt_matrices(xlfile):
    matrices = []
    for sheet in xlfile.sheet_names:
        if sheet != 'summary':
            df = xlfile.parse(sheet,skiprows=1)
            df.columns = df.columns.str.lower()
            df.columns = df.columns.str.strip()
            matrices = make_matrices(df, matrices)
        else:
            pass
    return(matrices)




pilot1_tt_matrices = tt_matrices(pilot1_tt)
pilot2_tt_matrices = tt_matrices(pilot2_tt)
pilot3_tt_matrices = tt_matrices(pilot3_tt)

tt_combined_matrices = combine_matrices(pilot1_tt_matrices, '1')
tt_combined_matrices = combine_matrices(pilot2_tt_matrices, '2', tt_combined_matrices)
tt_combined_matrices = combine_matrices(pilot3_tt_matrices, '3', tt_combined_matrices)


In [168]:
pilot1_UM = pd.ExcelFile(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\urinemarking\iwata_pilot_1_urine_marking_assay_with_threshold.xlsx")
pilot2_UM = pd.ExcelFile(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\urinemarking\iwata_pilot_2_urine_marking_assay_with_threshold.xlsx")
pilot3_UM = pd.ExcelFile(r"C:\Users\megha\Documents\Padilla-Coreano\DomComp\urinemarking\iwata_pilot_3_urine_marking_assay_with_threshold.xlsx")

def um_matrices_pilot1(xlfile):
    matrices = []
    for sheet in xlfile.sheet_names:
        if sheet.startswith('CAGE'):
            df = xlfile.parse(sheet)
            df.columns = df.columns.str.lower()
            df.columns = df.columns.str.strip()
            df = df[(df["ties"] != "TIE") & (df["ties"] != "tie")]
            matrices = make_matrices(df, matrices)
        else:
            pass
    return(matrices)

def um_matrices(xlfile):
    matrices = []
    for sheet in xlfile.sheet_names:
        if sheet.startswith('CAGE'):
            df = xlfile.parse(sheet,skiprows=1)
            df.columns = df.columns.str.lower()
            df.columns = df.columns.str.strip()
            df = df[(df["ties"] != "TIE") & (df["ties"] != "tie")]
            matrices = make_matrices(df, matrices)
        else:
            pass
    return(matrices)

um_pilot1_matrices = um_matrices_pilot1(pilot1_UM)
um_pilot2_matrices = um_matrices(pilot2_UM)
um_pilot3_matrices = um_matrices(pilot3_UM)

um_combined_matrices = combine_matrices(um_pilot1_matrices, '1')
um_combined_matrices = combine_matrices(um_pilot2_matrices, '2', um_combined_matrices)
um_combined_matrices = combine_matrices(um_pilot3_matrices, '3', um_combined_matrices)

um_combined_matrices

,*.1,*.2,*.3,*.4
1.1.1,0,1,1,0
1.1.2,1,0,0,1
1.1.3,0,0,0,0
1.1.4,0,1,2,0
1.2.1,0,2,1,1
...,...,...,...,...
3.5.4,1,1,0,0
3.6.1,0,0,0,0
3.6.2,0,0,0,0
3.6.3,1,1,0,1


In [171]:
with pd.ExcelWriter('linearity_matrices.xlsx') as writer:
    # Write DataFrame1 to a sheet named 'Sheet1'
    rc_combined_matrices.to_excel(writer, sheet_name='RC')

    # Write DataFrame2 to a sheet named 'Sheet2'
    hco_combined_matrices.to_excel(writer, sheet_name='HCO')

    tt_combined_matrices.to_excel(writer, sheet_name = 'TT')

    um_combined_matrices.to_excel(writer, sheet_name = 'UM')

 